# Predicting Housing Prices from User Preferences
## Shuhan Mao, Zhengzhi(Tim) Lu

## Introduction

In general terms, this project aims to help people who wish to buy a house of their preferences by predicting the price of that house that most fits their preferences and conditions (without considering housing price as a preference). For instance, if you are a person with at least $\$60,000$ annual income and prefer a house that is built within the last 50 years, this project will tell you the typical price of a house that is built within the last 50 years likely bought by other people with at least $\$60,000$ annual income. In other words, this project can, ideally, be treated as a price estimator of the houses that you want to buy.

The idea of predicting the price of a house based on user preferences originated from a conversation with classmates about choosing the best graduate department with a limited amount of money. We then realized that this idea could be more recognized if we include the general population as a part of our target user so as to now. The current sample data we use as a test for this project is the collection of residential blocks (or packs of houses divided by streets) throughout California, sourced from California Housing Prices from Kaggle (Nugent, 2020). However, for the current stage, this data sample is not specific enough to individual households (since samples are collected in the units of “blocks” rather than “houses”), so it is mainly used for testing the functionality of our project in general and not ready for public usage. For instance, the data sample contains “latitude” and “longitude”, which can be useful for realizing a price trend from a range of geographical locations as mathematical coordinates but is inappropriate for most users who refer to addresses instead.

The rest of the content will be covered as follows: First, we will introduce and explain the mathematical models we used for this project. Then, we will display our software implementation of the models. We will display the output from the software and evaluate the accuracy of this result. Last, we will touch on what can be done to improve the project upon our current progress.


## Mathematical Model
The mathematical models that we used for this project are primarily taken from a statistical point of view, where the concept of mean, regression, and probability are embodied in the models. In particular, the task of predicting the housing price based on user preferences is divided into two parts. The first part consists of training a program that can infer the most preferable houses to users based on the preferences they have indicated. The second part consists of training a program that predicts the price of a house based on housing features (location, house size, household median income, etc.) and the actual housing prices. Currently, the preferences we can accept from users must be those that are defined as a feature of the sample data for the second part of the task. 

### 1. Logistic Regression
The optimization problem we choose to describe for the first part is that of logistic regression. Here, we use the sigmoid function as the likelihood function for our problem and the Maximum Likelihood Estimation principle to set the objective function for our problem. In particular, the objective function is

$$\underset{\omega}{\max}\ \ \frac{1}{N}\sum_{i=1}^{N}ln\Big(\frac{1}{1+e^{-y_i\omega^T \Theta(x_i)}}\Big)$$

Where $y_i$ (equals to either 0 or 1) represents whether the $i^{th}$ sample satisfies (1) or doesn’t satisfy (0) user preference, $\Theta(x_i)$ represents the values of the $i^{th}$ sample’s features, and $N$ represents the number of samples used for training. $\omega$, the decision variable, represents the value of weights corresponding to the sample features. This part of the problem will be solved using gradient descent.

### 2. Quadratic Regression
The second part of the task uses quadratic regression (indicated as polynomial degree = 2 in the program next section) to train a function that fits with sample data. To fit the function as close to the sample data as possible, we minimize the mean squared error of the function, which is also our objective function

$$\underset{A}{\max}\ \ \frac{1}{N}\sqrt{\sum_{i=1}^{N}(y_i-A^T\Theta(x_i))^2}$$

Where $y_i$, a different $y_i$ from part 1, represents the actual housing price of the $i^{th}$ sample, $\Theta(x_i)$ represents the values of the $i^{th}$ sample’s features, and $N$ represents the number of samples used for training. $A$, the decision variable, represents the polynomial coefficients that best fit with sample data. This part of the program will be solved using CVX.

### 1 + 2.

To integrate the programs' tasks, we will first define a test preference to generate the choice filter $y$ that indicates whether a sample is within the range of preference and train the weights of the first program and polynomial coefficients of the second program. Then, we select a few samples that are most probable with the user preference (those with the highest $\omega^T\Theta(x_i)$ ) from the first program and feed them into the second program, which takes in their respective features as the input and produces the corresponding predicted house prices as the output. Macroscopically, this is as if the whole program predicts the housing price based on the user’s input preference.

# Solution

Note: the following code must be executed in its current order.

In [1]:
%%file generate_length.m
%helper function
function Q = generate_length(s,r)
    Q = 1;
    for g = 1:r
        Q = cat(1,Q,nchoosek(s,g));
    end
end

Created file '/Users/Tivager/generate_length.m'.


Define control parameters. 

For quadratic regression, assume polynomial_degree = 2. For mean square error, assume norm_degree = 2. 

number_top_choices = number of samples chosen that can obtain the highest probability, with largest $\ \omega^T\Theta(x_i)$.

In [2]:
clear; clc;
load('housing.mat');

%control parameters
polynomial_degree = 2;  norm_degree = 2;    train_data_ratio = 0.6;
step = 0.0001;  threshold = 1*10^-6;
number_top_choices = 100;

%another control parameter, preferences, is on line 60 and 61 since they
%have to be processed after the features are generated. In this example,
%preference indicates that a sample will be marked "1" if the median income
%for its residential block is greater than 50K, and marked "0" otherwise.

Import data sample, divide data into a train and a test set, and generate features for training.

In [3]:
%remove samples with NaNs. By inspection, only a minority of them are NaNs
%so hopefully this won't impact the data much.
housing = rmmissing(housing);
data = table2array(housing(:,1:9));
[samples, variables] = size(data);

%generate features for linear regression and logistic regression
input = data(:,1:variables-1);
output = data(:,variables);
length = generate_length(variables-1,polynomial_degree);
X = ones(samples,sum(length));
for i = 1:samples
    for k = 1:polynomial_degree
        X(i,1+sum(length(1:k)):sum(length(1:k+1))) = transpose(prod(nchoosek(input(i,:),k),2));
    end
end
B = randperm(size(X,1),floor(train_data_ratio*size(X,1)));
train_X1 = X(B,:);   train_Y1 = output(B,:);
test_X1 = X(setdiff(1:size(X,1),B),:);
test_Y1 = output(setdiff(1:size(output,1),B),:);
[samples, variables] = size(train_X1);
tests = i-samples;

Proprocess data for logistic regression so that most values are close to 1. Quadratic regression will also use this preprocessed data.

In [4]:
%preprocess data for logistic regression
for k = 1:variables
    train_X1(:,k) = train_X1(:,k)./abs(mean(train_X1(:,k)));
    test_X1(:,k) = test_X1(:,k)./abs(mean(test_X1(:,k)));
end
mean_Y = [abs(mean(train_Y1(:,1))), abs(mean(test_Y1(:,1)))];
train_Y1(:,1) = train_Y1(:,1)./abs(mean(train_Y1(:,1)));
test_Y1(:,1) = test_Y1(:,1)./abs(mean(test_Y1(:,1)));

Quadratic regression, part 2 of the program: cvx solves for parameters, $A$, that best fit the housing price of samples base on the featured data. In this case, data is quadratically featured.

Then display the mean square error to see how accurate is the model.

In [5]:
cvx_begin quiet
    variable A(sum(length));
    minimize norm((train_X1*A-train_Y1),norm_degree)./samples;
cvx_end
test_error = norm((test_X1*A-test_Y1),norm_degree)./tests;
disp("The mean square train error is " + cvx_optval + ". ");
disp("The mean square test error is " + test_error + ". ");
variables = variables - 1;  omega = zeros(variables,1); iterations = 0;
train_X1(:,1) = []; test_X1(:,1) = [];

The mean square train error is 0.0028359. 
The mean square test error is 0.0035387. 


### Define preferences: The example preference defined here states that samples with median income less than $\$30,000$ will be marked as 1, else marked as 0.

In [6]:
%generates preferences based on the input from a user, one of the control
%parameters
train_preference = double(data(B,8)<3); train_preference(train_preference==0) = -1;
test_preference = double(data(setdiff(1:size(X,1),B),8)<3);


%To set multiple preferences, use & operator on multiple unary conditions, such as

%train_preference = double(data(B,8)<10 & data(B,8)>3); train_preference(train_preference==0) = -1;
%test_preference = double(data(setdiff(1:size(X,1),B),8)<10 & data(setdiff(1:size(X,1),B),8)>3);

%which means to pick the samples with median income greater than $30,000 and less than $100,000.

Logistic Regression, part 1 of the program: use gradient descent to solve for the weights $\omega$.

Then display the training error as a percentage.

In [7]:
%use gradient descent to solve for optimal weights (omega) that
%characterize the preferences
while true
    gradient = -transpose(-sum(train_preference.*train_X1./(1+exp(train_preference.*(train_X1*omega))))./samples);
    if abs(sum(log(1+exp(-train_preference.*(train_X1*(omega+step*gradient)))))-sum(log(1+exp(-train_preference.*(train_X1*omega)))))/samples<=threshold
        break;
    end
    omega = omega+step*gradient;
    iterations = iterations + 1;
end

%check the accuracy of the weights
predictions_true = 1./(1+exp(-test_X1*omega))>0.5;
result = zeros(2,2,tests);
for sample = 1:tests
    result(:,:,sample) = [predictions_true(sample), ~predictions_true(sample)] & [test_preference(sample); ~test_preference(sample)];
end
result = sum(result,3)./sample;
disp("The error rate from logistic regression is " + (result(2,1) + result(1,2))*100 + "%. ");

The error rate from logistic regression is 7.5728%. 


Part 1 + 2: select the samples with highest probability from part 1 of the program and feed into part 2. Then display the predicted trpical housing price of these samples.

In [9]:
%choose number_top_choices(100 here) samples that best fit the
%input preferences and estimate the price of these samples using the
%regression model trained previously. Then compute the weighted average
%of these prices as the typical price a user has to pay for the house 
%they preferred.

collection = [1./(1+exp(-train_X1*omega)), train_X1; 1./(1+exp(-test_X1*omega)), test_X1];
collections = sortrows(collection,'descend');
collections(number_top_choices+1:end,:)=[];
for n = 1:number_top_choices
    prices(1,n) = [1, collections(n,2:end)]*A;
end
collections = [transpose(prices.*mean(mean_Y)), collections];
collections = sortrows(collections,'descend');
weighted_average_house_price = sum(collections(:,1).*collections(:,1))/sum(collections(:,1));
disp("The weighted average housing price is " + weighted_average_house_price +". ");

The weighted average housing price is 246822.0676. 


## Results and Discussion

The following tables display the predicted housing price outputs based on the preference on the median income of the household and the median age of the household. We generated the table of these two conditions for discusison because they can be understood and checked with enough intuition.


| Median income condition (in 10,000s) | Weighted average housing price |
| --- | --- |
|<3| 246822.0676|
|<5| 277461.4596|
|>3 & <5| 306121.988|
|<10| 309419.524|
|>3 & <10| 522561.1656|
|>3| 554956.4197|
|<10 & >5| 564106.4386|
|>5| 602766.0788|

| Median house age condition (in years) | Weighted average housing price |
| --- | --- |
|<50| 295425.8142|
|<40| 299730.6837|
|<30| 286676.9491|
|<20| 286126.2672|
|<10| 272668.9763|
|>10| 462874.092|
|>20| 368439.5427|
|>30| 323226.1166|
|>40| 199520.1253|
|>40 & <50| 116994.1465|
|>30 & <40| 114348.3159|
|>20 & <30| 135062.7932|
|>10 & <20| 282708.0834|
|>0 & <10| 275555.3968|
|>0 & <50| 297994.6153|


From the first table, we can infer that our trained model is at least accurate in the big picture; because people with higher income tend to have more money and can afford more expensive houses, the relation that having a higher median income correlates with having a higher housing price makes intuitive sense, and the programed managed to show this trend from the input preference (take the case of >3 & <5 and <10 & >5, for example). Also, comparing the value for conditions >3 & <10 and >3 (or <10 & >5 and >5), we see that the condition without the upper bound results in a higher housing price. It means that the weighted average housing price is higher when samples of higher median income are included, which also makes intuitive sense. However, since we assume we don't know any correlation among the feature of house prices, we can't make the assumption that any change in the condition boundary follows the intuition described above. On the other hand, we observe that the result degenerates when we apply the median income condition of >10 as this condition will result in 100K house price, far from >500K predicted with other similar conditions. One speculation to the cause of degeneration is that there are too few samples with median income greater than 100K that the training cannot determine many true positives from the false positives.

From the second table, we observe that the houses newer than 20 years are more expensive than the houses older than 20 years. We also observe that given a constant upper bound condition, the increase of the lower bound condition results in the decrease of weighted average house price. Both observations correspond to the relation that newer houses are generally more expensive than older houses. However, the converse doesn not match the relation: when given a constant lower bound condition, the increase of the upper bound condition should decrease the weighted average housing price since more samples of older houses are included, but the weighted average housing price actually increased a little. We do not know the cause of this discrepency with the previous observation, but this discrepency may imply that other features dependent to the median house age may have an inverse impact on the predicted housing price.

## Conclusion

In summary from the result, the program we trained can capture the macroscopic trend of the data and possibly predict the typical housing price based on a given preference. However, some trends cannot be intuitively explained (such as that from the second table). Therefore, we conclude that this program can be treated as a naive prototype for a "housing price estimator", not accurate enough for public usage. There are few ways to continue this project in the future, such as including samples of indiviual housing features (bedroom size, house size, etc.), training new models that generate features based on actual user input, and changing our current models (logistic regression to support vector machine/quadratic regression to cubic regression, for example). The first two ways allow more user decision input to the current program, and the last way may polish the flaws we have for our current program.

#### Citation:

Nugent, C., 2020. California Housing Prices. [online] Kaggle.com. Available at: <https://www.kaggle.com/camnugent/california-housing-prices>

